##### Master Degree in Computer Science and Data Science for Economics

# vLLM

### Sergio Picascia

[vLLM](https://github.com/vllm-project/vllm) is a fast and easy-to-use library for LLM inference and serving.

In [1]:
%pip install vllm

vLLM is an open-source library that offers a highly efficient and fast **serving engine** (it's not the model itself) for large language models (LLMs). It's designed to maximize the throughput (the amount of processed data over the infrastructure) of LLMs, which is a major challenge due to their large size and computational demands.

It achieves this speed-up by using a few key techniques:

* PagedAttention: This is the core innovation of vLLM. It is an attention algorithm inspired by virtual memory and paging in operating systems. It manages the Key-Value (KV) cache more efficiently by dividing it into fixed-size blocks. This prevents memory waste and allows for better resource utilization, especially when serving multiple requests in parallel with varying sequence lengths.

* Continuous Batching: Unlike traditional batching, which waits for all sequences in a batch to finish before starting a new one, continuous batching allows new sequences to enter the batch as soon as a sequence finishes. This keeps the GPU busy and improves overall throughput.

* Optimized CUDA Kernels: vLLM uses custom, highly optimized CUDA kernels to accelerate the attention calculation and other parts of the LLM pipeline

In [2]:
from vllm import LLM, SamplingParams
%pip install qwen_vl_utils #use Qwen llm
from qwen_vl_utils import process_vision_info
from transformers import AutoProcessor
import torch

INFO 08-31 14:07:08 [__init__.py:241] Automatically detected platform cuda.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00


In [3]:
DEVICE = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
MODEL_PATH = "Qwen/Qwen2.5-VL-7B-Instruct-AWQ"
DEVICE

'cuda:0'

In [ ]:
model = LLM(
    model=MODEL_PATH,
    enforce_eager=False,
    max_model_len=8192,
    #device=DEVICE, #auto detected internally
    gpu_memory_utilization=0.3,
    limit_mm_per_prompt={"video": 0, "image": 1}, #limits the number of multi-modal inputs per prompt
    max_num_batched_tokens=2048, #maximum number of tokens in a batch for processing
    max_num_seqs=64, #maximum number of sequences that can be processed in parallel
    enable_prefix_caching=True, #caching of prefixes for faster inference
    enable_chunked_prefill=True, #chunked prefill for better memory utilization.
)
processor = AutoProcessor.from_pretrained(MODEL_PATH, use_fast=True)
sampling_params = SamplingParams(
    temperature=0, #0 for deterministic sampling (greedy decoding), where the model always chooses the most likely next token.
    max_tokens=4096, #maximum number of tokens to generate in the output
)
processor

INFO 08-31 14:07:34 [utils.py:326] non-default args: {'model': 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ', 'max_model_len': 8192, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.3, 'max_num_batched_tokens': 2048, 'max_num_seqs': 64, 'disable_log_stats': True, 'limit_mm_per_prompt': {'video': 0, 'image': 1}, 'enable_chunked_prefill': True}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

INFO 08-31 14:08:11 [__init__.py:711] Resolved architecture: Qwen2_5_VLForConditionalGeneration
WARNING 08-31 14:08:11 [__init__.py:2768] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 08-31 14:08:11 [__init__.py:2819] Casting torch.bfloat16 to torch.float16.
INFO 08-31 14:08:11 [__init__.py:1750] Using max model len 8192
WARNING 08-31 14:08:14 [__init__.py:1171] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 08-31 14:08:14 [arg_utils.py:1770] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 08-31 14:08:14 [arg_utils.py:1569] --enable-prefix-caching is not supported for multimodal models in V0 and has been disabled.
INFO 08-31 14:08:15 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 08-31 14:08:15 [llm_engine.py:222] Initializing a V0 LLM engine (v0.10.1.1) wit

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

INFO 08-31 14:08:20 [cuda.py:384] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 08-31 14:08:20 [cuda.py:433] Using XFormers backend.
INFO 08-31 14:08:21 [parallel_state.py:1134] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
INFO 08-31 14:08:21 [model_runner.py:1080] Starting to load model Qwen/Qwen2.5-VL-7B-Instruct-AWQ...
INFO 08-31 14:08:22 [weight_utils.py:296] Using model weights format ['*.safetensors']


model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://redpandanetwork.org/get/files/image/galleries/28138502587_a0a020ae9a_k.jpeg",
                "min_pixels": 224 * 224,
                "max_pixels": 1280 * 28 * 28,
            },
            {"type": "text", "text": "Describe the image."},
        ],
    },
]

In [ ]:
prompt = processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
image_inputs, _ = process_vision_info(messages)
llm_inputs = {
    "prompt": prompt,
    "multi_modal_data": {"image": image_inputs},
}

In [ ]:
output = model.generate([llm_inputs], sampling_params=sampling_params)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it, est. speed input: 1117.17 toks/s, output: 99.24 toks/s]


In [ ]:
output[0].outputs[0].text

"The image shows a red panda, also known as a lesser panda or a red cat-bear, perched on a tree branch. The red panda has a distinctive coat with a mix of black, white, and reddish-brown fur. Its face is predominantly white with a black nose and a white muzzle. The red panda's ears are upright and rounded, and its eyes are dark and expressive. The background is a blurred green, suggesting a natural, forested environment. The red panda appears to be in a relaxed state, possibly observing its surroundings."